In [4]:
import numpy as np
import matplotlib.pyplot as plt
from functions import preprocess_raw_eeg
from scipy.fft import fft, fftfreq

# --- Load your raw EEG sample ---
# Replace this with your own npy file (4x250 or more)
raw_data = np.load(r"C:\Users\greym\Xavier\datasets\Grey\motor\left\1763064382661.npy")  # example path

fs = 250  # sampling rate (Hz)
n_channels, n_points = raw_data.shape
t = np.arange(n_points) / fs  # time vector (seconds)

# --- Run preprocessing from your pipeline ---
filtered_data, fft_data = preprocess_raw_eeg(
    data=raw_data.reshape(1, n_channels, n_points),  # shape = (samples, chans, points)
    fs=fs, lowcut=7, highcut=45, coi3order=0
)
filtered_data = filtered_data[0]  # remove batch dimension

# --- FFT for raw data (manual to compare) ---
freqs = fftfreq(n_points, 1 / fs)[: n_points // 2]
fft_raw = np.abs(fft(raw_data))[:, : n_points // 2]
fft_filtered = np.abs(fft(filtered_data))[:, : n_points // 2]

# =========================================
#        TIME-DOMAIN COMPARISON
# =========================================
plt.figure(figsize=(12, 8))
for ch in range(n_channels):
    plt.subplot(n_channels, 1, ch + 1)
    plt.plot(t, raw_data[ch], color="gray", alpha=0.6, label="Raw EEG")
    plt.plot(t, filtered_data[ch], color="blue", label="Filtered EEG", linewidth=1)
    plt.ylabel(f"Ch {ch+1}")
    plt.grid(True)
    if ch == 0:
        plt.legend(loc="upper right")
plt.xlabel("Time (s)")
plt.suptitle("EEG Filtering Visualization (Time Domain)")
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# =========================================
#        FREQUENCY-DOMAIN COMPARISON
# =========================================
plt.figure(figsize=(12, 8))
for ch in range(n_channels):
    plt.subplot(n_channels, 1, ch + 1)
    plt.plot(freqs, fft_raw[ch], color="gray", alpha=0.5, label="Raw Spectrum")
    plt.plot(freqs, fft_filtered[ch], color="purple", label="Filtered Spectrum")
    plt.xlim(0, 60)
    plt.ylabel(f"Ch {ch+1}")
    plt.grid(True)
    if ch == 0:
        plt.legend(loc="upper right")
plt.xlabel("Frequency (Hz)")
plt.suptitle("EEG Filtering Visualization (Frequency Domain)")
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# =========================================
#        DIFFERENCE TRACE (optional)
# =========================================
plt.figure(figsize=(12, 6))
for ch in range(n_channels):
    diff = raw_data[ch] - filtered_data[ch]
    plt.plot(t, diff + ch * 50, label=f"Ch {ch+1}")  # offset for clarity
plt.title("Difference (Raw - Filtered)")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude offset per channel")
plt.grid(True)
plt.legend()
plt.show()


Input data shape: (1, 8, 250)
         Removing channels with zero variance...
         New shape: (1, 7, 250)


BrainFlowError: INVALID_ARGUMENTS_ERROR:13 unable to apply band stop filter

In [9]:
import numpy as np
raw_data = np.load(r"C:\Users\greym\Xavier\datasets\Grey\motor\left\1763064382661.npy")
print("Raw shape:", raw_data.shape)
print("\n=== RAW ADC VALUES (UNSCALED) ===")
for i, ch in enumerate(raw_data):
    print(f"Ch {i+1}: mean={np.mean(ch):.4f}, std={np.std(ch):.4f}, min={np.min(ch):.2f}, max={np.max(ch):.2f}")

# Apply proper scaling to microvolts
SCALE_FACTOR = 0.02235174  # For Cyton: 4.5V / 24 (gain) / (2^23 - 1) * 1000000
scaled_data = raw_data * SCALE_FACTOR

print("\n=== SCALED TO MICROVOLTS (µV) ===")
for i, ch in enumerate(scaled_data):
    print(f"Ch {i+1}: mean={np.mean(ch):.2f}µV, std={np.std(ch):.2f}µV, min={np.min(ch):.2f}µV, max={np.max(ch):.2f}µV")


Raw shape: (8, 250)

=== RAW ADC VALUES (UNSCALED) ===
Ch 1: mean=-177.2219, std=1.0963, min=-179.35, max=-175.18
Ch 2: mean=-1583.5465, std=1.3084, min=-1586.20, max=-1581.15
Ch 3: mean=947.2589, std=2.0823, min=942.92, max=951.51
Ch 4: mean=-76.8679, std=1.1253, min=-79.15, max=-74.40
Ch 5: mean=4190.9512, std=0.0000, min=4190.95, max=4190.95
Ch 6: mean=834.2196, std=5.2898, min=820.86, max=844.25
Ch 7: mean=-504.2967, std=1.1679, min=-506.83, max=-501.75
Ch 8: mean=-1027.5952, std=1.2478, min=-1029.92, max=-1025.22

=== SCALED TO MICROVOLTS (µV) ===
Ch 1: mean=-3.96µV, std=0.02µV, min=-4.01µV, max=-3.92µV
Ch 2: mean=-35.40µV, std=0.03µV, min=-35.45µV, max=-35.34µV
Ch 3: mean=21.17µV, std=0.05µV, min=21.08µV, max=21.27µV
Ch 4: mean=-1.72µV, std=0.03µV, min=-1.77µV, max=-1.66µV
Ch 5: mean=93.68µV, std=0.00µV, min=93.68µV, max=93.68µV
Ch 6: mean=18.65µV, std=0.12µV, min=18.35µV, max=18.87µV
Ch 7: mean=-11.27µV, std=0.03µV, min=-11.33µV, max=-11.22µV
Ch 8: mean=-22.97µV, std=0.03µV, min

In [10]:
from brainflow import BoardShim, BoardIds
print(BoardShim.get_eeg_channels(BoardIds.CYTON_BOARD))


[1, 2, 3, 4, 5, 6, 7, 8]
